In [1]:
import json

FILE_PATH = "/home/jovyan/ColBERT/read_args.json"
with open(FILE_PATH, "r", encoding="utf-8") as json_file:
    # args = json.loads(f)
    json_args = json.load(json_file)

In [2]:
print(json_args)
print(json_args.__class__)

{'root': '/home/jovyan/ColBERT/read_outputs/', 'experiment': 'MSMARCO-psg', 'run': '2021-06-24_02.26.19', 'rank': -1, 'similarity': 'l2', 'dim': 128, 'query_maxlen': 32, 'doc_maxlen': 180, 'mask_punctuation': True, 'checkpoint': '/home/jovyan/ColBERT/train_outputs/MSMARCO-psg/train.py/msmarco.psg.l2/checkpoints/colbert-400000.dnn', 'bsize': 256, 'amp': True, 'queries': '/home/jovyan/ColBERT/data/test_queries.tsv', 'collection': '/home/jovyan/ColBERT/data/collection.tsv', 'qrels': None, 'index_root': '/home/jovyan/ColBERT/indexes/', 'index_name': 'MSMARCO.L2.32x200k', 'partitions': 32768, 'nprobe': 32, 'retrieve_only': False, 'faiss_name': None, 'faiss_depth': 1024, 'part_range': None, 'batch': False, 'depth': 1000, 'top_n_psg': 10}
<class 'dict'>


In [4]:
json_args.get("bsize")

256

In [6]:
import easydict
args = easydict.EasyDict(json_args)

In [7]:
for k, v in args.items():
    print(k, v)

root /home/jovyan/ColBERT/read_outputs/
experiment MSMARCO-psg
run 2021-06-24_02.26.19
rank -1
similarity l2
dim 128
query_maxlen 32
doc_maxlen 180
mask_punctuation True
checkpoint /home/jovyan/ColBERT/train_outputs/MSMARCO-psg/train.py/msmarco.psg.l2/checkpoints/colbert-400000.dnn
bsize 256
amp True
queries /home/jovyan/ColBERT/data/test_queries.tsv
collection /home/jovyan/ColBERT/data/collection.tsv
qrels None
index_root /home/jovyan/ColBERT/indexes/
index_name MSMARCO.L2.32x200k
partitions 32768
nprobe 32
retrieve_only False
faiss_name None
faiss_depth 1024
part_range None
batch False
depth 1000
top_n_psg 10


In [9]:
import os
import random

from colbert.utils.parser import Arguments
from colbert.utils.runs import Run

from colbert.evaluation.loaders import load_colbert, load_qrels, load_queries, load_collection_dict
from colbert.indexing.faiss import get_faiss_index_name
from colbert.reading.reader import read
from colbert.ranking.batch_retrieval import batch_retrieve


if args.collection is not None:
    args.collection_dict = load_collection_dict(args.collection)

if args.part_range:
    part_offset, part_endpos = map(int, args.part_range.split('..'))
    args.part_range = range(part_offset, part_endpos)

[Jun 24, 07:21:07] #> Loading collection...
0M 1M 2M 3M 4M 5M 6M 7M 8M 


In [12]:
import ujson

def load_colbert(args, do_print=True):
    from colbert.evaluation.load_model import load_model
    colbert, checkpoint = load_model(args, False)

    # TODO: If the parameters below were not specified on the command line, their *checkpoint* values should be used.
    # I.e., not their purely (i.e., training) default values.
    print("!!!!!"* 10)
    for k in ['query_maxlen', 'doc_maxlen', 'dim', 'similarity', 'amp']:
        if 'arguments' in checkpoint and hasattr(args, k):
            if k in checkpoint['arguments'] and checkpoint['arguments'][k] != getattr(args, k):
                a, b = checkpoint['arguments'][k], getattr(args, k)
                Run.warn(f"Got checkpoint['arguments']['{k}'] != args.{k} (i.e., {a} != {b})")

    if 'arguments' in checkpoint:
        if args.rank < 1:
            print(ujson.dumps(checkpoint['arguments'], indent=4))

    if do_print:
        print('\n')

    return colbert, checkpoint

In [14]:
colbert, checkpoint = load_colbert(args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[Jun 24, 07:24:19] checkpoint!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
{
    "root": "\/home\/jovyan\/ColBERT\/train_outputs\/",
    "experiment": "MSMARCO-psg",
    "run": "msmarco.psg.l2",
    "rank": 0,
    "similarity": "l2",
    "dim": 128,
    "query_maxlen": 32,
    "doc_maxlen": 180,
    "mask_punctuation": true,
    "resume": false,
    "resume_optimizer": false,
    "checkpoint": null,
    "lr": 3e-6,
    "maxsteps": 400000,
    "bsize": 32,
    "accumsteps": 1,
    "amp": true,
    "triples": "\/home\/jovyan\/ColBERT\/data\/triples.train.small.tsv",
    "queries": null,
    "collection": null
}




In [15]:
qrels = load_qrels(args.qrels)
queries = load_queries(args.queries)

[Jun 24, 07:24:29] #> Loading the queries from /home/jovyan/ColBERT/data/test_queries.tsv ...
[Jun 24, 07:24:29] #> Got 2 queries. All QIDs are unique.



In [16]:
args.index_path = os.path.join(args.index_root, args.index_name)

In [19]:
if args.faiss_name is not None:
    args.faiss_index_path = os.path.join(args.index_path, args.faiss_name)
else:
    args.faiss_index_path = os.path.join(args.index_path, get_faiss_index_name(args))

print(f"faiss_index_path: {args.faiss_index_path}")

faiss_index_path: /home/jovyan/ColBERT/indexes/MSMARCO.L2.32x200k/ivfpq.32768.faiss


In [22]:
from colbert.modeling.inference import ModelInference
from colbert.ranking.rankers import Ranker

# load colbert model and ready query/passage tokenizers
inference = ModelInference(colbert, amp=args.amp)
ranker = Ranker(args, inference, faiss_depth=args.faiss_depth)

[Jun 24, 07:28:06] #> Loading the FAISS index from /home/jovyan/ColBERT/indexes/MSMARCO.L2.32x200k/ivfpq.32768.faiss ..
[Jun 24, 07:28:16] #> Building the emb2pid mapping..
[Jun 24, 07:28:46] len(self.emb2pid) = 596847412
[Jun 24, 07:29:09] tensor.size() =  torch.Size([596847924, 128])
[Jun 24, 07:29:09] |> Loading /home/jovyan/ColBERT/indexes/MSMARCO.L2.32x200k/0.pt ...
[Jun 24, 07:29:11] |> Loading /home/jovyan/ColBERT/indexes/MSMARCO.L2.32x200k/1.pt ...
[Jun 24, 07:29:12] |> Loading /home/jovyan/ColBERT/indexes/MSMARCO.L2.32x200k/2.pt ...
[Jun 24, 07:29:14] |> Loading /home/jovyan/ColBERT/indexes/MSMARCO.L2.32x200k/3.pt ...
[Jun 24, 07:29:15] |> Loading /home/jovyan/ColBERT/indexes/MSMARCO.L2.32x200k/4.pt ...
[Jun 24, 07:29:17] |> Loading /home/jovyan/ColBERT/indexes/MSMARCO.L2.32x200k/5.pt ...
[Jun 24, 07:29:19] |> Loading /home/jovyan/ColBERT/indexes/MSMARCO.L2.32x200k/6.pt ...
[Jun 24, 07:29:20] |> Loading /home/jovyan/ColBERT/indexes/MSMARCO.L2.32x200k/7.pt ...
[Jun 24, 07:29:22

Process ForkPoolWorker-15:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Process ForkPoolWorker-8:
Process ForkPoolWorker-16:
KeyboardInterrupt
Process ForkPoolWorker-13:
Process ForkPoolWorker-11:
Process ForkPoolWorker-6:
Process ForkPoolWorker-14:
Process ForkPoolWorker-12:
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Process ForkPoolWorker-5:
Process ForkPoolWorker-7:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/root/conda/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/root/conda/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/root/conda/lib/python3.8/multip

In [23]:
from colbert.reading.extractive_reader import load_extractive_reader
reader_model, reader_tokenizer = load_extractive_reader(args)

In [116]:
import torch.nn.functional as F

def extractive_answer(question, text, model, tokenizer):

    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    # print(f"input_len: {len(input_ids)}")
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_start_prob = F.softmax(answer_start_scores, dim=-1)[:,answer_start]
    
    answer_end = torch.argmax(answer_end_scores)
    answer_end_prob = F.softmax(answer_end_scores, dim=-1)[:, answer_end]
    
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end+1])
    )
    return answer, answer_start, answer_start_prob, answer_end, answer_end_prob

In [117]:
import os
import time
import faiss
import random
import torch
import itertools

from colbert.reading.extractive_reader import load_extractive_reader
from colbert.utils.runs import Run
from multiprocessing import Pool
from colbert.modeling.inference import ModelInference
from colbert.evaluation.ranking_logger import RankingLogger

from colbert.utils.utils import print_message, batch
from colbert.ranking.rankers import Ranker


def read_seq(args, **kwargs):
    
    milliseconds = 0
    
    reader_model = kwargs.get("reader_model")
    reader_tokenizer = kwargs.get("reader_tokenizer")
    queries = kwargs.get("queries")
    
    qids_in_order = list(queries.keys())
    
    for qoffset, qbatch in batch(qids_in_order, 100, provide_offset=True):
        qbatch_text = [queries[qid] for qid in qbatch]

        rankings = []
        rankings_for_qa = []

        for query_idx, q in enumerate(qbatch_text):
            torch.cuda.synchronize('cuda:0')
            s = time.time()

            Q = ranker.encode([q])
            pids, scores = ranker.rank(Q)

            torch.cuda.synchronize()
            milliseconds += (time.time() - s) * 1000.0

            if len(pids):
                print(qoffset+query_idx, q, len(scores), len(pids), scores[0], pids[0],
                      milliseconds / (qoffset+query_idx+1), 'ms')

            rank_zip = zip(pids, scores)
            ranking, ranking_qa = itertools.tee(rank_zip)
            rankings.append(ranking)
            rankings_for_qa.append(ranking_qa)

        for query_idx, (qid, ranking) in enumerate(zip(qbatch, rankings)):
            query_idx = qoffset + query_idx

            if query_idx % 100 == 0:
                print_message(f"#> Logging query #{query_idx} (qid {qid}) now...")

            ranking = [(score, pid, None) for pid, score in itertools.islice(ranking, args.depth)]

        # reader
        print(len(rankings_for_qa))
        for i, (qid, ranking_qa) in enumerate(zip(qbatch, rankings_for_qa)):
            if qid in queries:
                query = queries[qid]                
                for j, (pid, score) in enumerate(itertools.islice(ranking_qa, args.top_n_psg)):
                    if str(pid) in args.collection_dict:
                        passage = args.collection_dict[str(pid)]
                        # print(i, score, pid, passage)
                        answer, answer_start, answer_start_prob, answer_end, answer_end_prob = extractive_answer(query, passage, reader_model, reader_tokenizer)
                        answer_prob = answer_start_prob * answer_end_prob
                        print(f"\n\nQuery: {query}")
                        print(f">> Passage [{j}][{score:.4f}]:\n{passage}")
                        print(f">> Answer: P[{answer_prob.item():.4f}]=S[{answer_start_prob.item():.4f}]E[{answer_end_prob.item():.4f}]\n{answer}")

In [118]:
read_seq(
    **{
        "args": args,
        "queries": queries,
        "reader_model": reader_model,
        "reader_tokenizer": reader_tokenizer,
    }
)

0 how long is a college hockey game 3033 3033 28.90803337097168 7306593 43.627023696899414 ms
1 Androgen receptor define 5649 5649 27.804271697998047 4339068 115.03255367279053 ms
[Jun 24, 09:51:06] #> Logging query #0 (qid 262156) now...
2


Query: how long is a college hockey game
>> Passage [0][28.9080]:
A: A college hockey game spans three 20-minute periods of regulation game time. This takes an average of two hours and 25 minutes to complete. Games tied after the final period extend into a five-minute sudden-death overtime period. If still tied after this, a shootout occurs to resolve the draw.
>> Answer: P[0.1312]=S[0.2846]E[0.4610]
a college hockey game spans three 20 - minute periods of regulation game time


Query: how long is a college hockey game
>> Passage [1][28.8719]:
A college hockey game spans three 20-minute periods of regulation game time. This takes an average of two hours and 25 minutes to complete. Games tied after the final period extend into a five-minute sudden-

In [104]:
tmp = queries

In [34]:
tmp.keys()

odict_keys([262156, 2])

In [66]:
def load_queries(queries_path):
    from collections import OrderedDict
    queries = OrderedDict()

    print_message("#> Loading the queries from", queries_path, "...")

    with open(queries_path, "r", encoding="utf-8") as f:
        for line in f:
            qid, query, *_ = line.strip().split('\t')
            qid = int(qid)

            assert (qid not in queries), ("Query QID", qid, "is repeated!")
            queries[qid] = query

    print_message("#> Got", len(queries), "queries. All QIDs are unique.\n")

    return queries

In [67]:
from collections import OrderedDict
custom_queries = OrderedDict()

In [119]:
custom_queries[0] = "what is process control equipment"
custom_queries[1] = "what is process improvement (lean six sigma)"
custom_queries[2] = "what is on the outside of dna"

In [121]:
read_seq(
    **{
        "args": args,
        "queries": custom_queries,
        "reader_model": reader_model,
        "reader_tokenizer": reader_tokenizer,
    }
)

0 what is process control equipment 6673 6673 27.03770637512207 7736238 64.70847129821777 ms
1 what is process improvement (lean six sigma) 6526 6526 29.331634521484375 7867876 131.25181198120117 ms
2 what is on the outside of dna 6400 6400 26.231773376464844 7186309 149.91609255472818 ms
[Jun 24, 09:56:19] #> Logging query #0 (qid 0) now...
3


Query: what is process control equipment
>> Passage [0][27.0377]:
Terms used for Process Control and Monitoring Equipment and Systems. Basic Process Control System (BPCS): the entire control system described in this specification.
>> Answer: P[0.1623]=S[0.4374]E[0.3710]
basic process control system


Query: what is process control equipment
>> Passage [1][26.3383]:
Process equipment is equipment used in chemical and materials processing, in facilities like refineries, chemical plants, and wastewater treatment plants. This equipment is usually designed with a specific process or family of processes in mind and can be customized for a particular 